In [20]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from astropy.io import ascii

### Loading scaler and MLP

In [21]:
with open('Blazhko-BLG-StandardScaler.pkl', 'rb') as fid:
    clf_sca = pickle.load(fid)
    
with open('MLP-Blazhko-BLG.pkl', 'rb') as fid:
    clf_cla = pickle.load(fid)

### Photometric data from the OGLE survey for the Galactic bulge

In [58]:
# http://www.astrouw.edu.pl/ogle/ogle4/OCVS/blg/rrlyr/RRab.dat
# --- with replaced "-" by 0.0

data = ascii.read("RRab.dat")

## Header ## 
## ID I V Per ePer M0 Amp R21 phi21 R31 phi31 ##

per = data["col4"]
amp = data["col7"]
r21 = data["col8"]
phi21 = data["col9"]
r31 = data["col10"]
phi31 = data["col11"]

test_sample = np.vstack([per, amp, r21, phi21, r31, phi31]).T

### MLP Magic

In [59]:
X_test_scaled = clf_sca.transform(test_sample)
y_scoreMLP = clf_cla.predict(X_test_scaled)
y_scoreMLP_proba = clf_cla.predict_proba(X_test_scaled)

In [63]:
%matplotlib

plt.scatter(r31[np.where(y_scoreMLP == 0)], phi21[np.where(y_scoreMLP == 0)], 
            s=1,zorder=-100, c="C3", label="Modulated-class")
plt.scatter(r31[np.where(y_scoreMLP == 1)], phi21[np.where(y_scoreMLP == 1)], 
            s=1, c="C0", label="Non-modulated-class")
plt.legend()
plt.show()

Using matplotlib backend: Qt5Agg


### Writting stuff

In [95]:

data["Class"] = y_scoreMLP
data["ClassProba_0"] = y_scoreMLP_proba[:,0]
data["ClassProba_1"] = y_scoreMLP_proba[:,1]
data.write('RRab-classified.dat', overwrite=True, format="ascii") 